# Edmunds Data Acquisation

In [181]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
import string 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json

## 1. Fetch at least 5,000 posts from Edmunds
The following are the 3 candidate forems we are considering

In [2]:
midsize_sedan_url = "https://forums.edmunds.com/discussion/7526/general/x/midsize-sedans-2-0"
rental_cars_url = "https://forums.edmunds.com/discussion/9125/general/x/rental-car-experiences"
entry_level_lux_url = "https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans"

Information to be extracted:
    date, userid, and message

### Midsize Sedan

In [4]:
page_1 = requests.get(midsize_sedan_url)
page_1_text = page_1.text
page_1_soup = BeautifulSoup(page_1_text, 'html.parser')

In [6]:
page_1_soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">

 PAGESETUP = window.PAGESETUP||{};
 PAGESETUP.scope = PAGESETUP.scope || {};
 PAGESETUP.scope.environmentEdwURL = "https://edw.edmunds.com";
 PAGESETUP.scope.environmentURL = "https://www.edmunds.com";

</script><title>Midsize Sedans 2.0 — Car Forums at Edmunds.com</title>
<link href="https://forums.edmunds.com/discussion/7526/general/x/midsize-sedans-2-0/p2" rel="next"/>
<link href="/applications/dashboard/design/style.css?v=61f18917" media="all" rel="stylesheet" static="1"/>
<link href="/applications/dashboard/design/style-compat.css?v=61f18917" media="all" rel="stylesheet" static="1"/>
<link href="/plugins/badges/design/badges.css?v=61f18917" media="all" rel="stylesheet" static="1"/>
<link href="/resources/design/vanillicon.css?v=61f18917" media="all" rel="stylesheet" static="1"/>
<link href="/plugins/editor/design/editor.css?v=61f1

Explore the html structure in order to target the location of data we are interested in.

User ID

In [31]:
page_1_soup.find_all(class_='Comment')[0].find(class_='Author').find("a").get("data-userid")

'410384'

Comment String

In [33]:
page_1_soup.find_all(class_='Comment')[0].find(class_='Message userContent').text

'\nHi Pat:You forgot the Chrysler Sebring  '

In [75]:
test_s = page_1_soup.find_all(class_='Comment')[0].find(class_='Message userContent').text
word_tokenize(test_s)

['Hi', 'Pat', ':', 'You', 'forgot', 'the', 'Chrysler', 'Sebring']

Date

In [39]:
page_1_soup.find_all(class_='Comment')[0].find(class_='Permalink').find("time").get("title")

'April 11, 2007 6:52PM'

Comment ID

In [51]:
page_1_soup.find_all(class_='Comment')[0].find(class_='Permalink').get("href").split('/')[-1].split('_')[-1]

'3504465'

In [41]:
len(page_1_soup.find_all(class_='Comment'))

50

Note that one page contains 50 comments, for 5000 comments we need to extract info from at least 100 pages.

In [102]:
# Helper function that extract desired information from 1 comment.
def extract_comment_info(item):
    comment_id = item.find(class_='Permalink').get("href").split('/')[-1].split('_')[-1]
    user_id = item.find(class_='Author').find("a").get("data-userid")
    date = item.find(class_='Permalink').find("time").get("title")
    comment = item.find(class_='Message userContent').text
    #remove newline character 
    comment_mod = comment.replace("\n","")
    return [comment_id,user_id,date,comment_mod]
        

Extract info from page 1.

In [105]:
comments_pg1 = page_1_soup.find_all(class_='Comment')
comment_id_lst = []
user_id_lst = []
date_lst = []
comment_lst = []
for i in comments_pg1:
    comment_info = extract_comment_info(i)
    comment_id_lst.append(comment_info[0])
    user_id_lst.append(comment_info[1])
    date_lst.append(comment_info[2])
    comment_lst.append(comment_info[3])

In [106]:
pd.DataFrame({'comment_id':comment_id_lst,'user_id':user_id_lst,'date':date_lst,'comment':comment_lst}).head(50)

,comment_id,user_id,date,comment
0,3504465,410384,"April 11, 2007 6:52PM",Hi Pat:You forgot the Chrysler Sebring
1,3515400,209396,"April 11, 2007 7:33PM",I'm sure some folks would appreciate having th...
2,3516719,457562,"April 12, 2007 6:51AM",You can try to revive this topic but without b...
3,3517791,410027,"April 12, 2007 8:43AM",Model vs. model is exactly what we're here for...
4,3518875,411850,"April 13, 2007 11:49AM",The Altima is my favorite of the bunch. It is ...
5,3519952,410865,"April 13, 2007 12:05PM",What do you think of the CVT in the Altima? I'...
6,3521034,410821,"April 13, 2007 12:18PM",Buick LaCrossePassat(Audi A6 in non-lux trim)V...
7,3522123,411850,"April 13, 2007 12:24PM","I have actually driven the CVT, and honestly i..."
8,3523194,380418,"April 13, 2007 12:27PM","I wouldn't consider the RX-8, G35 coupe, or Sa..."
9,3504466,380418,"April 13, 2007 12:32PM","My daily driver is an '03 Maxima, and the '07 ..."


In [107]:
page_url_str = '/p'
# Helper function that extract desired information from page.
def extract_n_pages_comments(url, n_pages, tokenize_flg = False):
    if tokenize_flg:
        comments_dict = {'page':[],'comment_id':[],'user_id':[],'date':[],'comment':[],'comment_tokenized':[]}
    else:
        comments_dict = {'page':[],'comment_id':[],'user_id':[],'date':[],'comment':[]}
    for n in range(n_pages):
        pg_n = n+1
        if pg_n == 1:
            page_str = ''
        else:
            page_str = page_url_str + str(pg_n)
        print("Currently working on page:", pg_n)
        page = requests.get(url+page_str)
        page_text = page.text
        page_soup = BeautifulSoup(page_text, 'html.parser')
        page_comments = page_soup.find_all(class_='Comment')
        page_comment_id_lst = []
        page_user_id_lst = []
        page_date_lst = []
        page_comment_lst = []
        if tokenize_flg:
            page_comment_t_lst = []
        for comment in page_comments:
            page_comment_info = extract_comment_info(comment)
            page_comment_id_lst.append(page_comment_info[0])
            page_user_id_lst.append(page_comment_info[1])
            page_date_lst.append(page_comment_info[2])
            page_comment_lst.append(page_comment_info[3])
            if tokenize_flg:
                page_comment_t_lst.append(word_tokenize(page_comment_info[3]))
        page_lst = [str(pg_n)]*len(page_comment_id_lst)
        comments_dict['page'] += page_lst
        comments_dict['comment_id'] += page_comment_id_lst
        comments_dict['user_id'] += page_user_id_lst
        comments_dict['date'] += page_date_lst
        comments_dict['comment'] += page_comment_lst
        if tokenize_flg:
            comments_dict['comment_tokenized'] += page_comment_t_lst
    return comments_dict

In [117]:
comments = extract_n_pages_comments(midsize_sedan_url, 150,tokenize_flg = True)

Currently working on page: 1
Currently working on page: 2
Currently working on page: 3
Currently working on page: 4
Currently working on page: 5
Currently working on page: 6
Currently working on page: 7
Currently working on page: 8
Currently working on page: 9
Currently working on page: 10
Currently working on page: 11
Currently working on page: 12
Currently working on page: 13
Currently working on page: 14
Currently working on page: 15
Currently working on page: 16
Currently working on page: 17
Currently working on page: 18
Currently working on page: 19
Currently working on page: 20
Currently working on page: 21
Currently working on page: 22
Currently working on page: 23
Currently working on page: 24
Currently working on page: 25
Currently working on page: 26
Currently working on page: 27
Currently working on page: 28
Currently working on page: 29
Currently working on page: 30
Currently working on page: 31
Currently working on page: 32
Currently working on page: 33
Currently working o

In [112]:
comments.keys()

dict_keys(['page', 'comment_id', 'user_id', 'date', 'comment', 'comment_tokenized'])

Check point: Save as csv

In [118]:
pd.DataFrame(dict((key,value) for key, value in comments.items() if key != 'comment_tokenized')).to_csv('comments_raw.csv',index=False)

## 2. Remove stop words

Remove punctuations and stop words.

In [177]:
stemmer = PorterStemmer()
stop_words = stopwords.words('english')
lst_of_tokenized = comments['comment_tokenized']
for sentence in lst_of_tokenized:
    new_sentence = []
    for s in sentence:
        if s in string.punctuation or s == '..':
            sentence.remove(s)
        if s in stop_words:
            sentence.remove(s)

## 3. Tagging Tokens

In [178]:
remove_lst = ['CC','CD','DT','EX','IN','LS','PDT','PRP','POS','RP','VBP','VBZ','TO','UH','WDT','WP','WP$','WRB']
def remove_unwanted_words(tagged_sentence):
    new_sentence = []
    for word_tuple in tagged_sentence:
        if word_tuple[1] not in remove_lst:
            new_sentence.append(word_tuple)
    return new_sentence

In [179]:
lst_tagged_sentences = []

for sentence in lst_of_tokenized:
    tagged_lst = nltk.pos_tag(sentence)
    tagged_lst_filtered = remove_unwanted_words(tagged_lst)
    lst_tagged_sentences.append(tagged_lst_filtered)

In [180]:
lst_tagged_sentences[4]

[('Altima', 'NNP'),
 ('favorite', 'JJ'),
 ('bunch', 'NN'),
 ('fastest', 'JJS'),
 ('best', 'JJS'),
 ('handling', 'VBG'),
 ('group', 'NN'),
 ('priced', 'VBN'),
 ('well', 'RB'),
 ('model', 'NN'),
 ('best', 'RBS'),
 ('looker', 'NN'),
 ('bunch', 'NN'),
 ('IMO', 'NNP'),
 ('My', 'NNP'),
 ('second', 'JJ'),
 ('choice', 'NN'),
 ('Ford', 'NNP'),
 ('Fusion', 'NNP'),
 ('things', 'NNS'),
 ("n't", 'RB'),
 ('lack', 'VB'),
 ('punch', 'JJ'),
 ('gadget-free', 'JJ'),
 ('compared', 'VBN'),
 ('Altima', 'NNP'),
 ('still', 'RB'),
 ('fun', 'RBR'),
 ('drive', 'JJ'),
 ('nice', 'JJ'),
 ('looking', 'VBG'),
 ('Camry', 'NNP'),
 ('interior', 'JJ'),
 ('Accord', 'NNP'),
 ('seats', 'NNS'),
 ('Camry', 'NNP'),
 ('also', 'RB'),
 ('lot', 'NN'),
 ('slower', 'JJR'),
 ('HP', 'NNP'),
 ('rating', 'NN'),
 ('Nissan', 'NNP'),
 ('motor', 'NN'),
 ('Accord', 'NNP'),
 ('whiny', 'NN'),
 ('V6', 'NNP'),
 ('Both', 'NNP'),
 ('Camry', 'NNP'),
 ('Accord', 'NNP'),
 ('kind', 'NN'),
 ('expensive', 'JJ'),
 ('get', 'NN'),
 ('Aura', 'NNP'),
 ('neat

In [182]:
comments['tagged_filtered_sentence'] = lst_tagged_sentences

In [183]:
comments.keys()

dict_keys(['page', 'comment_id', 'user_id', 'date', 'comment', 'comment_tokenized', 'tagged_filtered_sentence'])

Check point: JSON file

In [194]:
with open('comments_cleaned.json', 'w') as fp:
    json.dump(comments, fp)

Load JSON file

In [188]:
with open('comments_cleaned.json', 'r') as fp:
    loaded_data = json.load(fp)

In [191]:
loaded_data.keys()

dict_keys(['page', 'comment_id', 'user_id', 'date', 'comment', 'comment_tokenized', 'tagged_filtered_sentence'])

In [195]:
loaded_data['comment_tokenized'][4]

['The',
 'Altima',
 'favorite',
 'bunch',
 'It',
 'amongst',
 'fastest',
 'best',
 'handling',
 'group',
 'priced',
 'well',
 'gadgets',
 'The',
 '2007',
 'model',
 'best',
 'looker',
 'bunch',
 'IMO',
 'My',
 'second',
 'choice',
 'Ford',
 'Fusion',
 'The',
 'things',
 'I',
 "n't",
 'like',
 'lack',
 'punch',
 'gadget-free',
 'compared',
 'Altima',
 'It',
 'still',
 'fun',
 'drive',
 'nice',
 'looking',
 'inside',
 'though',
 'I',
 'hate',
 'Camry',
 "'s",
 'interior',
 'Accord',
 "'s",
 'seats',
 'The',
 'Camry',
 'also',
 'feels',
 'lot',
 'slower',
 'HP',
 'rating',
 'suggests',
 'Nissan',
 "'s",
 'motor',
 'wails',
 'Accord',
 "'s",
 'whiny',
 'V6',
 'Both',
 'Camry',
 'Accord',
 'kind',
 'expensive',
 'get',
 'The',
 'Aura',
 'looks',
 'neat',
 'I',
 "'ve",
 'never',
 'driven',
 'one',
 'I',
 'ca',
 "n't",
 'comment',
 'The',
 'Mazda',
 'needs',
 'replaced',
 'Not',
 'bad',
 'car',
 'generationally',
 'feels',
 'like',
 '2002',
 'car',
 '2007',
 'I',
 "'ll",
 'take',
 'host',
 "'

In [198]:
loaded_data['tagged_filtered_sentence'][4]

[['Altima', 'NNP'],
 ['favorite', 'JJ'],
 ['bunch', 'NN'],
 ['fastest', 'JJS'],
 ['best', 'JJS'],
 ['handling', 'VBG'],
 ['group', 'NN'],
 ['priced', 'VBN'],
 ['well', 'RB'],
 ['model', 'NN'],
 ['best', 'RBS'],
 ['looker', 'NN'],
 ['bunch', 'NN'],
 ['IMO', 'NNP'],
 ['My', 'NNP'],
 ['second', 'JJ'],
 ['choice', 'NN'],
 ['Ford', 'NNP'],
 ['Fusion', 'NNP'],
 ['things', 'NNS'],
 ["n't", 'RB'],
 ['lack', 'VB'],
 ['punch', 'JJ'],
 ['gadget-free', 'JJ'],
 ['compared', 'VBN'],
 ['Altima', 'NNP'],
 ['still', 'RB'],
 ['fun', 'RBR'],
 ['drive', 'JJ'],
 ['nice', 'JJ'],
 ['looking', 'VBG'],
 ['Camry', 'NNP'],
 ['interior', 'JJ'],
 ['Accord', 'NNP'],
 ['seats', 'NNS'],
 ['Camry', 'NNP'],
 ['also', 'RB'],
 ['lot', 'NN'],
 ['slower', 'JJR'],
 ['HP', 'NNP'],
 ['rating', 'NN'],
 ['Nissan', 'NNP'],
 ['motor', 'NN'],
 ['Accord', 'NNP'],
 ['whiny', 'NN'],
 ['V6', 'NNP'],
 ['Both', 'NNP'],
 ['Camry', 'NNP'],
 ['Accord', 'NNP'],
 ['kind', 'NN'],
 ['expensive', 'JJ'],
 ['get', 'NN'],
 ['Aura', 'NNP'],
 ['neat